In [25]:
import json
import gradio as gr
import requests
import matplotlib.pyplot as plt


def request_iris_prediction(data_list):
    endpoint = "http://5990e067-635c-4263-aa56-7cc55f42d72f.koreacentral.azurecontainer.io/score"
    headers = {
        'Content-Type':'application/json',
        'Authorization':''
    }

    body = {
        "Inputs": {
            "input1": data_list
        }
    }

    response = requests.post(endpoint, headers=headers, json=body)
    
    if response.status_code == 200:
        response_json = response.json()
        return response_json["Results"]["WebServiceOutput0"]
    else:
        return list()

def save_plot(data_points):
    # 센터로이드의 평균 위치를 계산하기 위한 변수 초기화
    centroid_positions = {0: [0, 0], 1: [0, 0], 2: [0, 0]}
    centroid_colors = {0: 'b', 1: 'r', 2: 'g'}  # 클러스터 색상
    
    # 데이터 포인트를 기반으로 센터로이드 위치 계산
    for point in data_points:
        assignment = point["Assignments"]
    
        # 각 클러스터별로 거리 데이터 가져오기
        for i in range(3):
            dist_key = f"DistancesToClusterCenter no.{i}"
            if dist_key in point:
                # 위치의 평균 계산
                centroid_positions[i][0] += (point["sepal_length_cm"] + point[dist_key]) / 2
                centroid_positions[i][1] += (point["sepal_width_cm"] + point[dist_key]) / 2
    
    # 평균값으로 센터로이드 위치 계산
    for i in range(3):
        centroid_positions[i][0] /= len(data_points)
        centroid_positions[i][1] /= len(data_points)
    
    plt.figure(figsize=(8, 6))
    
    point_index = 0
    # 데이터 포인트 그리기
    for point in data_points:
        point_index += 1
        plt.scatter(point["sepal_length_cm"], point["sepal_width_cm"],
                    c='b' if point["Assignments"] == 0 else 'r' if point["Assignments"] == 1 else 'g')
        plt.text(point["sepal_length_cm"], point["sepal_width_cm"], f"{point_index}")

    # 클러스터 센터로이드 그리기
    for cluster, (x, y) in centroid_positions.items():
        plt.scatter(x, y, c=centroid_colors[cluster], marker='X', s=200)

    plt.title('Data Points and Cluster Centroids')
    plt.xlabel('Sepal Length (cm)')
    plt.ylabel('Sepal Width (cm)')
    plt.grid()
    plt.savefig('iris_clusters.png')
    plt.close()  # plt.show() 대신 plt.close()를 사용
    return 'iris_clusters.png'  # 현재 figure 반환


with gr.Blocks() as demo:
    
    view_count = gr.State(1)
    data_dict = dict()
    
    def click_add(count):
        count += 1
        print(count)
        return count
        
    def click_delete(count):
        if count > 1:
            count -= 1
        print(count)
        return count
        
    def click_send():
        data_list = list(data_dict.values())
        response_data = request_iris_prediction(data_list=data_list)
        save_plot(response_data)
        return response_data, "iris_clusters.png"
        
    def change_data(i, sl, sw, pl, pw):
        data_dict.update({
            i: {
                "sepal_length_cm": sl,
                "sepal_width_cm": sw,
                "petal_length_cm": pl,
                "petal_width_cm": pw,
                "class": "unknown"
            }
        })
        print(data_dict)
 
        
    gr.Markdown("### 붓꽃 예측")
    
    with gr.Row():
        add_button = gr.Button("+")
        delete_button = gr.Button("-")
        
    # 데이터 입력 필드 추가 필요
    with gr.Column():
        @gr.render(inputs=[view_count])
        def render_input_components(count):
            
            for i in range(0, count):
                
                with gr.Column():
                    gr.Markdown(f"Index: {i}")
                    row_index = gr.State(i)
                    with gr.Row():
                        sepal_length_number = gr.Number(label="꽃받침 길이", key=f"sepal-length-{i}", value="")
                        sepal_width_number = gr.Number(label="꽃받침 너비", key=f"sepal-width-{i}", value="")
                        petal_length_number = gr.Number(label="꽃잎 길이", key=f"petal-length-{i}", value="")
                        petal_width_number = gr.Number(label="꽃잎 너비", key=f"petal-width-{i}", value="")
                        
                        sepal_length_number.change(fn=change_data, inputs=[row_index,
                            sepal_length_number, sepal_width_number, petal_length_number, petal_width_number],
                            outputs=[])
                        
                        sepal_width_number.change(fn=change_data, inputs=[row_index,
                            sepal_length_number, sepal_width_number, petal_length_number, petal_width_number],
                            outputs=[])
                        
                        petal_length_number.change(fn=change_data, inputs=[row_index,
                            sepal_length_number, sepal_width_number, petal_length_number, petal_width_number],
                            outputs=[])
                        
                        petal_width_number.change(fn=change_data, inputs=[row_index,
                            sepal_length_number, sepal_width_number, petal_length_number, petal_width_number],
                            outputs=[])
    
    result_textbox = gr.Textbox(label="결과")
    result_image = gr.Image(label="결과 이미지", type="filepath")
    send_button = gr.Button("전송")
    
    #add 이벤트
    #delete 이벤트
    #전송 버튼에 대한 이벤트
    add_button.click(fn=click_add, inputs=[view_count], outputs=[view_count])
    delete_button.click(fn=click_delete, inputs=[view_count], outputs=[view_count])
    send_button.click(fn=click_send, inputs=[], outputs=[result_textbox, result_image])
    
    
demo.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


{0: {'sepal_length_cm': '', 'sepal_width_cm': '', 'petal_length_cm': '', 'petal_width_cm': '', 'class': 'unknown'}}
{0: {'sepal_length_cm': '', 'sepal_width_cm': '', 'petal_length_cm': '', 'petal_width_cm': '', 'class': 'unknown'}}
{0: {'sepal_length_cm': '', 'sepal_width_cm': '', 'petal_length_cm': '', 'petal_width_cm': '', 'class': 'unknown'}}
{0: {'sepal_length_cm': '', 'sepal_width_cm': '', 'petal_length_cm': '', 'petal_width_cm': '', 'class': 'unknown'}}
{0: {'sepal_length_cm': 6, 'sepal_width_cm': '', 'petal_length_cm': '', 'petal_width_cm': '', 'class': 'unknown'}}
{0: {'sepal_length_cm': 6, 'sepal_width_cm': 4, 'petal_length_cm': '', 'petal_width_cm': '', 'class': 'unknown'}}
{0: {'sepal_length_cm': 6, 'sepal_width_cm': 4, 'petal_length_cm': 2, 'petal_width_cm': '', 'class': 'unknown'}}
{0: {'sepal_length_cm': 6, 'sepal_width_cm': 4, 'petal_length_cm': 2, 'petal_width_cm': 3, 'class': 'unknown'}}
2
{0: {'sepal_length_cm': 6, 'sepal_width_cm': 4, 'petal_length_cm': 2, 'petal_wid